In [1]:
import numpy
import scipy.special
import glob
import imageio
import cv2
import os
import random
from tkinter import *
import PIL
import pickle
from PIL import ImageGrab

In [2]:
class neuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        #number of input nodes
        self.input_nodes = inputnodes
        #number of hidden nodes
        self.hidden_nodes = hiddennodes
        #number of output nodes
        self.output_nodes = outputnodes
        #learning rate
        self.lr = learningrate
        
        
        #random weight of links between input and hidden layer (in range of -0.5 to +0.5)
        self.weight_input_hidden = (numpy.random.rand(self.hidden_nodes, self.input_nodes) - 0.5)
        #random weight of links between hidden and output layer (in range of -0.5 to +0.5)
        self.weight_hidden_output = (numpy.random.rand(self.output_nodes, self.hidden_nodes) - 0.5)
        
        
        #sigmoid function
        #def relu(x):
   # return max(0, x)
        #self.activation_function = lambda x: scipy.special.expit(x) 
        
    def activation_function(self,x):
        return 1 / (1 + numpy.e ** -x)
    
    def train(self, inputs_list, targets_list):
        
        #convert inputs and targets list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        #inputs to hidden layer
        hidden_inputs = numpy.dot(self.weight_input_hidden, inputs)
        #outputs from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #inputs to output layer
        final_inputs = numpy.dot(self.weight_hidden_output, hidden_outputs)
        #outputs from output layer
        final_outputs = self.activation_function(final_inputs)
        
        #error between target value and observed value of the output layer
        output_errors = targets - final_outputs
        
        #error for the hidden layer via backpropagation
        hidden_errors = numpy.dot(self.weight_hidden_output.T, output_errors)
        
        #gradient descent to update the weights
        #update the weights of the links between hidden and output layer
        self.weight_hidden_output += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        #update the weights of the links between the input and hidden layer
        self.weight_input_hidden += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
    
    def predict(self, inputs_list):
        
        #convert input list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        #inputs to hidden layer
        hidden_inputs = numpy.dot(self.weight_input_hidden, inputs)
        
        #outputs from the hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #inputs to output layer
        final_inputs = numpy.dot(self.weight_hidden_output, hidden_outputs)
        
        #outputs from the output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    
    def numbers_to_strings(self,argument):
        return { 
            0: "क", 
            1: "ख", 
            2: "ग",
            3: "घ",
            4: "ङ ",
            5: "च ",
            6: "छ ",
            7: "ज ",
            8: "झ ",
            9: "ञ",
            10: "ट",
            11: "ठ",
            12: "ड",
            13: "ढ",
            14: "ण",
            15: "त",
            16: "थ",
            17: "द",
            18: "ध",
            19: "न",
            20: "प",
            21: "फ",
            22:"ब",
            23: "भ",
            24:"म",
            25:"य",
            26:"र",
            27:"ल",
            28:"व",
            29:" श",
            30: "ष",
            31: "स",
            32:"ह",
            33:"क्ष",
            34:"त्र",
            35:"ज्ञ",
        }[argument]

In [3]:
input_nodes = 1024 
hidden_nodes = 340
output_nodes = 36
learning_rate = 0.1

In [ ]:
neural = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)


In [ ]:
#load the save model
saveneural=pickle.load(file=open("saveneuralnetworkobj.pkl","rb"))

In [ ]:
class main:
    def __init__(self, master):
        self.master = master
        self.res = ""
        self.pre = [None, None]
        self.bs = 10
        self.c = Canvas(self.master,bd=3,relief="ridge", width=320, height=320, bg='white')
        self.c.pack(side=LEFT)
        f1 = Frame(self.master, padx=5, pady=5)
        Label(f1,text="ANN Predicting  Nepali Hand-Written Character",fg="green",font=("",15,"bold")).pack(pady=10)
      
        Label(f1,text="Using Python ",fg="green",font=("",15)).pack()
        Label(f1,text="Draw On The Canvas Alongside",fg="green",font=("",15)).pack()
        self.pr = Label(f1,text="Prediction: None",fg="blue",font=("",20,"bold"))
        self.pr.pack(pady=20)
        
        Button(f1,font=("",15),fg="white",bg="red", text="Clear Canvas", command=self.clear).pack(side=BOTTOM)

        f1.pack(side=RIGHT,fill=Y)
        self.c.bind("<Button-1>", self.putPoint)
        self.c.bind("<ButtonRelease-1>",self.getResult)
        self.c.bind("<B1-Motion>", self.paint)

    def getResult(self,e):
        x = self.master.winfo_rootx() + self.c.winfo_x()
        y = self.master.winfo_rooty() + self.c.winfo_y()
        x1 = x + self.c.winfo_width()
        y1 = y + self.c.winfo_height()
        img = PIL.ImageGrab.grab()
        img = img.crop((x, y, x1, y1))
        img.save("dist.png")
        for image_file_name in glob.glob("dist.png"):
            print ("loading ... ", image_file_name)
            img_array = imageio.imread(image_file_name, as_gray=True)
            img_array = cv2.resize(img_array, (32,32))
            img_data  = 255.0 - img_array.reshape(1024)
            img_data = (img_data / 255.0 * 0.99) + 0.01
            image_array_ = numpy.asfarray(img_data).reshape((32,32))
    
            outputs = saveneural.predict(img_data)
            for n in range(output_nodes):
                print(neural.numbers_to_strings(n)+"===>"+str(outputs[n]))
            label = numpy.argmax(outputs)
            nepali_character= neural.numbers_to_strings(label)
        
        self.pr['text'] = "Prediction: " + nepali_character

    def clear(self):
        self.c.delete('all')

    def putPoint(self, e):
        self.c.create_oval(e.x - self.bs, e.y - self.bs, e.x + self.bs, e.y + self.bs, outline='black', fill='black')
        self.pre = [e.x, e.y]

    def paint(self, e):
        self.c.create_line(self.pre[0], self.pre[1], e.x, e.y, width=self.bs * 2, fill='black', capstyle=ROUND,
                           smooth=TRUE)

        self.pre = [e.x, e.y]


if __name__ == "__main__":
    root = Tk()
    main(root)
    root.title('Nepali Character Classifier')
    root.resizable(0, 0)
    root.mainloop()


loading ...  dist.png
क===>[0.0001444]
ख===>[8.35709743e-08]
ग===>[0.00808409]
घ===>[1.42266315e-10]
ङ ===>[0.01411466]
च ===>[9.32716989e-05]
छ ===>[3.99129571e-06]
ज ===>[6.35741443e-07]
झ ===>[5.76988885e-10]
ञ===>[5.31569841e-06]
ट===>[5.96256135e-05]
ठ===>[0.13460966]
ड===>[0.00320102]
ढ===>[1.34445764e-06]
ण===>[4.83654317e-05]
त===>[0.75032581]
थ===>[6.25665044e-06]
द===>[0.27780748]
ध===>[2.34157516e-07]
न===>[2.78346246e-07]
प===>[0.82666179]
फ===>[0.00327683]
ब===>[2.55869636e-07]
भ===>[6.61525883e-08]
म===>[2.57280136e-05]
य===>[0.00057672]
र===>[0.0006931]
ल===>[1.631121e-07]
व===>[2.01187491e-07]
 श===>[0.00152761]
ष===>[1.27785397e-05]
स===>[0.00058459]
ह===>[5.00998305e-05]
क्ष===>[1.62672708e-08]
त्र===>[0.0002456]
ज्ञ===>[1.98523186e-06]
loading ...  dist.png
क===>[0.0001444]
ख===>[8.35709743e-08]
ग===>[0.00808409]
घ===>[1.42266315e-10]
ङ ===>[0.01411466]
च ===>[9.32716989e-05]
छ ===>[3.99129571e-06]
ज ===>[6.35741443e-07]
झ ===>[5.76988885e-10]
ञ===>[5.31569841e-06]
ट

loading ...  dist.png
क===>[1.58197094e-05]
ख===>[0.00623212]
ग===>[0.00564706]
घ===>[0.03639057]
ङ ===>[0.10459898]
च ===>[0.00039339]
छ ===>[0.00041222]
ज ===>[7.65982362e-06]
झ ===>[4.44424742e-06]
ञ===>[0.00513614]
ट===>[1.42246252e-06]
ठ===>[0.00021439]
ड===>[0.01404292]
ढ===>[3.8934954e-09]
ण===>[0.03994897]
त===>[0.01306693]
थ===>[0.00045235]
द===>[6.42347309e-05]
ध===>[0.00014681]
न===>[6.70964366e-06]
प===>[0.00358487]
फ===>[0.00101414]
ब===>[0.00015513]
भ===>[7.34979482e-06]
म===>[0.00728378]
य===>[0.00032672]
र===>[4.09600232e-05]
ल===>[7.24950237e-05]
व===>[0.00019966]
 श===>[0.00773819]
ष===>[0.02357527]
स===>[0.01843108]
ह===>[8.0681931e-05]
क्ष===>[0.00014293]
त्र===>[2.94183109e-05]
ज्ञ===>[1.25474714e-07]
loading ...  dist.png
क===>[1.8905778e-06]
ख===>[0.05334088]
ग===>[0.00626704]
घ===>[0.00656651]
ङ ===>[0.01507693]
च ===>[5.79017737e-05]
छ ===>[0.00134701]
ज ===>[1.72096783e-05]
झ ===>[3.03072624e-07]
ञ===>[0.07847514]
ट===>[3.25651992e-05]
ठ===>[6.29023824e-05]
ड=

loading ...  dist.png
क===>[0.00583885]
ख===>[2.63687061e-06]
ग===>[0.02232013]
घ===>[1.16223798e-10]
ङ ===>[0.12858521]
च ===>[4.56698883e-06]
छ ===>[0.00019466]
ज ===>[4.95927359e-05]
झ ===>[1.76765696e-08]
ञ===>[0.00025249]
ट===>[1.38410543e-05]
ठ===>[0.01939905]
ड===>[4.81029401e-06]
ढ===>[3.36779665e-06]
ण===>[4.97684221e-06]
त===>[0.00296473]
थ===>[7.44077947e-07]
द===>[0.15793099]
ध===>[1.52393349e-06]
न===>[8.60141638e-06]
प===>[0.05251373]
फ===>[0.00475937]
ब===>[6.17655106e-05]
भ===>[1.34158328e-08]
म===>[0.03606825]
य===>[0.00279348]
र===>[0.00018089]
ल===>[2.45820212e-08]
व===>[5.27100978e-07]
 श===>[0.01487335]
ष===>[1.66037917e-06]
स===>[0.00037093]
ह===>[0.00044207]
क्ष===>[5.10731411e-06]
त्र===>[0.00659185]
ज्ञ===>[1.43439317e-05]
loading ...  dist.png
क===>[0.0069804]
ख===>[1.35223432e-05]
ग===>[0.00033191]
घ===>[0.00010053]
ङ ===>[0.09253323]
च ===>[3.35702743e-06]
छ ===>[9.63578902e-07]
ज ===>[5.43042396e-05]
झ ===>[1.17781493e-05]
ञ===>[3.44382096e-06]
ट===>[1.7316

loading ...  dist.png
क===>[0.03894176]
ख===>[1.66633865e-05]
ग===>[0.02609774]
घ===>[0.00033542]
ङ ===>[0.09050066]
च ===>[2.08746137e-06]
छ ===>[1.49783344e-06]
ज ===>[5.98359072e-06]
झ ===>[1.15292832e-05]
ञ===>[3.61879744e-06]
ट===>[3.60310949e-06]
ठ===>[0.0018599]
ड===>[2.54917273e-05]
ढ===>[7.21626891e-07]
ण===>[0.00022248]
त===>[0.0021609]
थ===>[9.84543333e-05]
द===>[0.00743883]
ध===>[0.00018923]
न===>[2.15285552e-05]
प===>[0.00012146]
फ===>[0.00025485]
ब===>[0.0013631]
भ===>[6.38927691e-05]
म===>[0.0491497]
य===>[0.01228533]
र===>[6.06070535e-05]
ल===>[1.81526602e-05]
व===>[0.00040684]
 श===>[0.03910021]
ष===>[0.00059068]
स===>[0.00239556]
ह===>[0.00011061]
क्ष===>[3.07975821e-06]
त्र===>[0.00352518]
ज्ञ===>[1.20408671e-05]
loading ...  dist.png
क===>[0.0273785]
ख===>[1.91304706e-07]
ग===>[0.14501835]
घ===>[8.54855978e-10]
ङ ===>[0.00325571]
च ===>[0.00318196]
छ ===>[4.29794558e-06]
ज ===>[0.01227803]
झ ===>[3.53498682e-08]
ञ===>[0.00010061]
ट===>[4.2802393e-06]
ठ===>[6.8288154

loading ...  dist.png
क===>[0.00057149]
ख===>[2.05442979e-06]
ग===>[0.00017825]
घ===>[8.22122233e-10]
ङ ===>[0.27979555]
च ===>[5.38515894e-05]
छ ===>[1.06829621e-05]
ज ===>[0.00135067]
झ ===>[0.02704851]
ञ===>[0.01649509]
ट===>[1.25103151e-05]
ठ===>[0.00013774]
ड===>[1.33967566e-06]
ढ===>[3.54584758e-07]
ण===>[0.00045276]
त===>[2.51227894e-06]
थ===>[4.42710676e-07]
द===>[0.00795717]
ध===>[2.46066399e-05]
न===>[2.01990259e-06]
प===>[5.4425779e-05]
फ===>[0.00018084]
ब===>[2.6531775e-05]
भ===>[4.16682349e-05]
म===>[0.00029528]
य===>[1.36687441e-05]
र===>[4.40584876e-05]
ल===>[1.12558504e-05]
व===>[1.94467182e-07]
 श===>[0.00050848]
ष===>[5.3138436e-07]
स===>[1.58667848e-05]
ह===>[0.00835883]
क्ष===>[1.38782725e-05]
त्र===>[0.07437221]
ज्ञ===>[0.00053295]
loading ...  dist.png
क===>[0.00022049]
ख===>[5.42643602e-06]
ग===>[0.00996943]
घ===>[2.19352778e-07]
ङ ===>[0.13495111]
च ===>[1.57925626e-05]
छ ===>[0.0021886]
ज ===>[0.00061765]
झ ===>[0.00030427]
ञ===>[0.01036441]
ट===>[8.48394535e-0

...

loading ...  dist.png
क===>[0.00011083]
ख===>[0.0024636]
ग===>[0.00025945]
घ===>[2.18453141e-06]
ङ ===>[0.25979852]
च ===>[0.04046088]
छ ===>[5.50293341e-05]
ज ===>[0.03183919]
झ ===>[6.4419192e-05]
ञ===>[4.42815426e-05]
ट===>[2.13308409e-07]
ठ===>[0.00018074]
ड===>[0.00210358]
ढ===>[1.17132519e-06]
ण===>[1.03144465e-05]
त===>[6.40191753e-06]
थ===>[5.38986797e-05]
द===>[4.29269511e-05]
ध===>[0.00095723]
न===>[0.00056846]
प===>[0.00017283]
फ===>[7.9560198e-05]
ब===>[0.05649426]
भ===>[1.62868137e-06]
म===>[0.00030585]
य===>[7.49892906e-05]
र===>[0.00139604]
ल===>[1.31682147e-05]
व===>[0.00016677]
 श===>[2.47396473e-05]
ष===>[2.2678181e-05]
स===>[0.00043405]
ह===>[0.00063079]
क्ष===>[5.90601227e-08]
त्र===>[0.00018605]
ज्ञ===>[1.03337948e-05]
loading ...  dist.png
क===>[1.44046731e-05]
ख===>[1.6550601e-06]
ग===>[1.70139657e-07]
घ===>[3.9747039e-07]
ङ ===>[0.34122173]
च ===>[0.43126886]
छ ===>[0.00162886]
ज ===>[6.05807772e-09]
झ ===>[0.00228673]
ञ===>[3.57534072e-06]
ट===>[3.06015488e-08]

loading ...  dist.png
क===>[0.0001336]
ख===>[0.01235502]
ग===>[0.00406632]
घ===>[1.90032464e-05]
ङ ===>[0.00046976]
च ===>[5.18576148e-05]
छ ===>[3.14981602e-07]
ज ===>[3.697845e-06]
झ ===>[0.0143011]
ञ===>[0.01685016]
ट===>[3.31228051e-05]
ठ===>[3.99368006e-05]
ड===>[1.25325233e-06]
ढ===>[1.57844901e-07]
ण===>[0.00966154]
त===>[0.00016716]
थ===>[0.01396648]
द===>[3.22516354e-07]
ध===>[0.00238468]
न===>[0.00429852]
प===>[1.29844317e-05]
फ===>[0.00035195]
ब===>[3.71094339e-06]
भ===>[0.00063907]
म===>[0.00291918]
य===>[0.00174074]
र===>[3.10092766e-06]
ल===>[2.05390059e-07]
व===>[3.12492785e-05]
 श===>[0.02381528]
ष===>[0.00034009]
स===>[0.00095342]
ह===>[0.00013435]
क्ष===>[0.00200174]
त्र===>[9.2031442e-05]
ज्ञ===>[6.70595744e-07]
loading ...  dist.png
क===>[2.12370366e-06]
ख===>[0.00361471]
ग===>[0.00144805]
घ===>[3.30404881e-05]
ङ ===>[0.00164449]
च ===>[2.13878641e-05]
छ ===>[3.27561753e-07]
ज ===>[5.6220813e-06]
झ ===>[0.00341111]
ञ===>[0.00089786]
ट===>[4.55070046e-05]
ठ===>[1.719

loading ...  dist.png
क===>[3.82567599e-07]
ख===>[9.68418544e-07]
ग===>[0.00048577]
घ===>[3.34870765e-08]
ङ ===>[7.48272041e-07]
च ===>[0.06082938]
छ ===>[5.61642261e-06]
ज ===>[0.00922102]
झ ===>[4.64861545e-05]
ञ===>[0.00523926]
ट===>[7.2570976e-06]
ठ===>[1.61937285e-05]
ड===>[0.0008864]
ढ===>[3.20951203e-05]
ण===>[0.00013427]
त===>[0.06510743]
थ===>[2.45662401e-06]
द===>[0.00600559]
ध===>[2.14100227e-05]
न===>[3.78160426e-07]
प===>[0.96103348]
फ===>[0.00040381]
ब===>[1.1516195e-06]
भ===>[7.71778571e-09]
म===>[1.30086061e-06]
य===>[1.00985384e-05]
र===>[7.91370465e-07]
ल===>[3.87597854e-10]
व===>[0.00178906]
 श===>[1.32011323e-10]
ष===>[1.46893874e-08]
स===>[4.17213261e-06]
ह===>[5.96756652e-08]
क्ष===>[2.52233522e-12]
त्र===>[0.01013514]
ज्ञ===>[6.03272089e-08]
loading ...  dist.png
क===>[3.07488504e-05]
ख===>[5.43577419e-05]
ग===>[1.65712172e-07]
घ===>[6.6721229e-07]
ङ ===>[3.97267585e-05]
च ===>[0.01108968]
छ ===>[0.00013375]
ज ===>[0.00128248]
झ ===>[0.00754731]
ञ===>[2.3688367e-

loading ...  dist.png
क===>[2.48735387e-09]
ख===>[4.04315617e-08]
ग===>[9.20937913e-05]
घ===>[1.08098092e-07]
ङ ===>[1.37886486e-07]
च ===>[0.31948509]
छ ===>[6.32386036e-07]
ज ===>[0.00030149]
झ ===>[5.178202e-10]
ञ===>[0.0024381]
ट===>[0.00170074]
ठ===>[1.85466435e-05]
ड===>[0.00025524]
ढ===>[8.65358477e-08]
ण===>[0.00305899]
त===>[0.00881004]
थ===>[7.48228657e-05]
द===>[0.00063094]
ध===>[3.99135224e-05]
न===>[9.4107512e-07]
प===>[0.85587893]
फ===>[3.56920724e-05]
ब===>[1.91035747e-11]
भ===>[5.27857354e-09]
म===>[2.35856176e-08]
य===>[1.4814236e-05]
र===>[0.00653946]
ल===>[4.51661599e-07]
व===>[0.00013175]
 श===>[1.19510069e-06]
ष===>[2.12762121e-11]
स===>[4.11348276e-05]
ह===>[1.78229137e-08]
क्ष===>[2.92626383e-08]
त्र===>[8.64312469e-07]
ज्ञ===>[8.26252228e-07]
loading ...  dist.png
क===>[5.70556807e-08]
ख===>[5.95849361e-08]
ग===>[3.86446283e-05]
घ===>[5.13762856e-08]
ङ ===>[1.66644256e-06]
च ===>[0.018774]
छ ===>[6.88755455e-05]
ज ===>[0.00020198]
झ ===>[6.52628123e-11]
ञ===>[2.

loading ...  dist.png
क===>[1.33739767e-07]
ख===>[5.28698588e-06]
ग===>[0.00374604]
घ===>[5.96126093e-08]
ङ ===>[1.10607603e-06]
च ===>[0.49293892]
छ ===>[1.25504751e-05]
ज ===>[0.00181772]
झ ===>[1.11950389e-05]
ञ===>[0.00031827]
ट===>[1.28954801e-09]
ठ===>[1.10164399e-06]
ड===>[0.00018745]
ढ===>[9.96749329e-08]
ण===>[4.81407727e-05]
त===>[0.00069445]
थ===>[2.24176515e-06]
द===>[0.00210508]
ध===>[3.47829618e-06]
न===>[7.51867916e-08]
प===>[0.95545087]
फ===>[4.9637264e-05]
ब===>[4.55268734e-06]
भ===>[1.54320609e-06]
म===>[1.81213433e-05]
य===>[0.0001532]
र===>[4.14658622e-07]
ल===>[5.87779832e-08]
व===>[7.18526098e-06]
 श===>[1.02376097e-09]
ष===>[9.00155345e-09]
स===>[3.93515704e-07]
ह===>[4.17575167e-07]
क्ष===>[1.37709232e-10]
त्र===>[0.03481165]
ज्ञ===>[1.32409576e-06]
loading ...  dist.png
क===>[7.90823933e-07]
ख===>[0.01398281]
ग===>[1.23720652e-06]
घ===>[1.09587877e-06]
ङ ===>[0.0030832]
च ===>[0.00340557]
छ ===>[0.00149067]
ज ===>[0.32027852]
झ ===>[0.00151619]
ञ===>[4.04189747